# 해커톤 1122
다양한 데이터를 연동해서 운전자를 어시스트한다.
예시:
- 차량 조작법
    - 자기 차량 또는 랜트카 혹은 대리 기사는 차량 조작이 미숙하다. 예를 들어 주유구 여는 방법, 열선 시트 조작법 등등.
- 차량 상태 조회
    - 주행 가능거리, 주유 필요 여부, 경고등 확인 등
    - 블루 링크에 연동된 차량 (현대 기아차 제외 확인 필요함)
    - 공통차량 api를 통해서 사용자 차량 정보를 가져올수 있음


## Setup

In [2]:
# 참고: 최신 버전의 OpenAI Python 라이브러리를 설치 및/또는 업그레이드를 권장합니다.
# %pip install --upgrade openai
# 최신버전 openai-1.3.4 에서 function 호출시 이슈 있음. 일부 함수 deprecated 예상
# 환경은 우선 python 3.9.2로 수행
%pip install openai==0.28

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 0. 환경 변수 설정

In [4]:
import openai
import json
import os

from dotenv import load_dotenv
load_dotenv()

# Azure OpenAI resource 정보를 설정합니다.
openai.api_type     = os.getenv("OPENAI_API_TYPE")
openai.api_key      = os.getenv("OPENAI_API_KEY")
openai.api_base     = os.getenv("OPENAI_API_BASE")
openai.api_version  = os.getenv("OPENAI_API_VERSION")   # API 버전은 "2023-07-01-preview" 부터 사용 가능합니다.
# deployment_id       = os.getenv("DEPLOYMENT_NAME")      # Azure OpenAI 리소스의 배포 이름입니다. gpt-35-turbo (0613) 또는 gpt-4 (0613) 버전 이상부터 지원합니다.
# deployment_id       = "gpt-4-1106-preview"
deployment_id       = "gpt-35-turbo-16k"

## 1. Function Call 설정
호출 할수 있는 available 함수 설정 필요

In [24]:
def get_function_call(messages, function_call = "auto"):
    # Define the functions to use
    functions = [
        {
            "name": "get_gasoline_expense",
            "description": "Get my car gasoline expense",
            "parameters": {
                "type": "object",
                "properties": {
                    "model_name": {
                        "type": "string",
                        "description": "Car model. If you cannot get model name please ask",
                    }
                },
                "required": ["model"],
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = openai.ChatCompletion.create(
        deployment_id = deployment_id,
        messages=messages,
        functions=functions,
        function_call=function_call, 
        temperature=0
    )
    return response

### 1.1 주행 가능 거리 조회

In [25]:
class DrivingRange:
    def __init__(self, timestamp: str, value: int, unit: str) -> None:
        self.timestamp = timestamp
        self.value = value
        self.unit = unit

def get_driving_range(id: int = 1): 
    # see https://developers.kia.com/web/v1/kia/specification/data/status_dte
    return DrivingRange(
        timestamp = "20231122",
        value = 360,
        unit = "km"
    )

### 1.2 공통 차량 정보 조회

In [26]:
class CarInfo:
    def __init__(self, id: int, fuel_eco: float, fuel_tank: int, fuel: str) -> None:
        self.id = id
        self.fuel_eco = fuel_eco
        self.fuel_tank = fuel_tank
        self.fuel = fuel


def get_car_info(model_name: str): 
    # call vehicle-web here
    return CarInfo(
        id = 1,
        fuel_eco = 15.0,
        fuel = "DIESEL",
        fuel_tank = 50
    )

### 1.3 유가 정보 조회

In [27]:
class FuelPrice:
    def __init__(self, category: str, price: float) -> None:
        self.category = category
        self.price = price

def match_fuel_category(category: str, fuelPrice: FuelPrice):
    return fuelPrice.category == category

def get_today_fuel_price(category: str):
    # call external-api-hub
    response = [
        FuelPrice(
            category = "REGULAR_GASOLINE",
            price = 1659.96
        ),
        FuelPrice(
            category = "DIESEL",
            price = 1607.84
        )
    ]

    result = [p for p in response if  match_fuel_category(category, p)][0]
    return result.price;

# print(get_today_fuel_price(category = "DIESEL"))

## 2. Function에 available 함수 설정

In [28]:
# 주유비 계산
def get_gasoline_expense(model_name: str):
     car_info = get_car_info(model_name)
     driving_range = get_driving_range(car_info.id)
     fuel_price = get_today_fuel_price(category = car_info.fuel)

     print(("driving_range is %i %s and fuel_eco is %f") % (driving_range.value, driving_range.unit, car_info.fuel_eco))
     current_fuel = driving_range.value / car_info.fuel_eco
     print(("current_fuel is %f") % (current_fuel))
     fuel_consumption = car_info.fuel_tank - current_fuel
     print(("fuel_consumption is %f") % (fuel_consumption))
     assert fuel_consumption >= 0

     return fuel_consumption * fuel_price

# print(("gasoline_expense is %.2f") % (get_gasoline_expense()))

## 3. 호출 테스트

In [35]:
first_message = [{"role": "system", "content": "you are driving assistant. You never answer you dont know"}, {"role": "user", "content": "내 차 주유하면 얼마나 나와?"}]
# 'auto' : Let the model decide what function to call
print("주유비 조회 테스트:")
first_response = get_function_call(first_message, "auto")
print(first_response)
print(first_response["choices"][0]["message"]["content"])

assert "function_call" not in first_response["choices"][0]["message"]

first_message.append(
    {
        "role": first_response["choices"][0]["message"]["role"],
        "content": first_response["choices"][0]["message"]["content"]
    }
)

first_message.append({"role": "user", "content": "모델은 카니발이야"})
second_response = get_function_call(first_message, "auto")
# print(response)

assert "function_call" in second_response["choices"][0]["message"]
print(second_response["choices"][0]["message"]["function_call"]["name"])
print(second_response["choices"][0]["message"]["function_call"]["arguments"])

available_functions = {
    "get_gasoline_expense": get_gasoline_expense
}

function_call_name = second_response["choices"][0]["message"]["function_call"]["name"]
fuction_to_call = available_functions[function_call_name]
function_args = json.loads(second_response["choices"][0]["message"]["function_call"]["arguments"])
function_response = fuction_to_call(**function_args)
print(function_response)

first_message.append(
    {
        "role": second_response["choices"][0]["message"]["role"],
        "name": second_response["choices"][0]["message"]["function_call"]["name"],
        "content": second_response["choices"][0]["message"]["function_call"]["arguments"],
    }
)

first_message.append(
    {
        "role": "function",
        "name": function_call_name,
        "content": function_response.__str__,
    }
)

response = openai.ChatCompletion.create(
    messages = first_message,
    deployment_id = deployment_id
)
print(response["choices"][0]["message"]["content"])



주유비 조회 테스트:
{
  "id": "chatcmpl-8NW66xBoua2DWyofzHEUmuW6fylEL",
  "object": "chat.completion",
  "created": 1700615618,
  "model": "gpt-4",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {}
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "\ucc28 \ubaa8\ub378 \uc774\ub984\uc744 \uc54c\ub824\uc8fc\uc2dc\uba74 \uc8fc\uc720 \ube44\uc6a9\uc744 \uacc4\uc0b0\ud574 \ub4dc\ub9b4 \uc218 \uc788\uc2b5\ub2c8\ub2e4. \ucc28 \ubaa8\ub378 \uc774\ub984\uc744 \uc54c\ub824\uc8fc\uc2dc\uaca0\uc5b4\uc694?"
      },
      "content_filter_result": {
        "error": {
          "code": "content_filter_error",
          "message": "The contents are not filtered"
        }
      },
      "content_filter_results": {}
    }
  ],
  "usage": {
    "prompt_tokens": 90,
    "completion_tokens": 50,
    "total_tokens": 140
  },
  "system_fingerprint": "fp_50a4261de5"
}
차 모델 이

## csv 업로드 검토

In [12]:
# 추천

# Read the CSV file
import pandas as pd
data = pd.read_csv('./data/parking_review_score9.csv')

def get_parking_suggest(model_name: str = ""):
    messages = [
        {
            "role": "system",
            "content": """
                You are assistant of parking service. You only response result as json.
                Dataset contains content, score, provider_name, and name.

                First filter data by name matches input model name.

                Then you suggest provider_name from filterd dataset by positive content and higher score.
                The score is from 1 to 5 and 5 is max.
                And the suggestion is effected by content more.

                And find most positive content from filtered dataset as reason.

                Expansive is negative.

                The json example is
                {
                    "model_name": "카니발",
                    "provider_name": "주차장",
                    "reason": "넓다"
                }

                If you cannot found the provider_name sample:
                {
                    "model_name": "카니발",
                    "provider_name": null,
                    "reason": "데이터가 없음"
                }
            """
        },
        {
            "role": "user",
            "content": ("""
                아래 데이터에서 %s인 차량이 제일 선호하는 주차장 1개만 그리고 이유를 알려줘
                %s
            """) % (model_name, data.to_string)
        }
    ]

    response = openai.ChatCompletion.create(
        deployment_id = deployment_id,
        messages=messages,
        temperature=0,
        max_tokens=10000
    )

    # print(response)
    # print(response["choices"][0]["message"]["content"])

    return response["choices"][0]["message"]["content"]

get_parking_suggest("아반떼")

'{\n    "model_name": "아반떼",\n    "provider_name": "판교 공영주차장",\n    "reason": "저렴하게 이용가능해서 좋습니다"\n}'